# 1-A description of the problem and a discussion of the background.

Taking a look at a bus stop in Toronto city, located at latitude = 43.649815 and longitude = -79.342163, we can see that there aren’t many places to eat. However, there are quite a few places that are related to sports such as gyms, Baseball fields and a lake for rowing. After doing some kind of exercise, we tend to get hungry. But the only options available are Coffee shops.



# 2-A description of the data and how it will be used to solve the problem


The data is based of foursquare API explore feature, listing nearby spots to a Toronto bus stop. Taking into consideration that we use a radius of 800, there are 11 spots near the bus station. Only two of which are food related, a Tim Hortons, a famous Canadian Coffee shop, and a MountainView Fare trade Coffee. Using this information, the idea is to open a new restaurant that athletes can have a meal after a long workout session. 

In [65]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library

print('Folium installed')
print('Libraries imported.')

Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.

Folium installed
Libraries imported.


In [66]:
CLIENT_ID = 'DRZ1AHJ2TXIZH5GHWBR15XSQ2B42VBVCCJGEEYDYP2J3EJAB' # your Foursquare ID
CLIENT_SECRET = '5FKJIHAC31FA3D54GW4DH25YSFXHB35NDQCKUQ5UW3MR2CBA' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: DRZ1AHJ2TXIZH5GHWBR15XSQ2B42VBVCCJGEEYDYP2J3EJAB
CLIENT_SECRET:5FKJIHAC31FA3D54GW4DH25YSFXHB35NDQCKUQ5UW3MR2CBA


In [67]:
latitude = 43.649815
longitude = -79.342163

In [68]:
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, 800, LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?client_id=DRZ1AHJ2TXIZH5GHWBR15XSQ2B42VBVCCJGEEYDYP2J3EJAB&client_secret=5FKJIHAC31FA3D54GW4DH25YSFXHB35NDQCKUQ5UW3MR2CBA&ll=43.649815,-79.342163&v=20180604&radius=800&limit=30'

In [69]:
import requests

In [70]:
results = requests.get(url).json()
'There are {} near the bus stop.'.format(len(results['response']['groups'][0]['items']))

'There are 11 near the bus stop.'

In [71]:
items = results['response']['groups'][0]['items']
items[0]

{'reasons': {'count': 0,
  'items': [{'summary': 'This spot is popular',
    'type': 'general',
    'reasonName': 'globalInteractionReason'}]},
 'venue': {'id': '52adedf011d24be20f5c4a8f',
  'name': 'BATLgrounds',
  'location': {'address': '33 Villiers St',
   'crossStreet': 'Villiers St. & Cherry St.',
   'lat': 43.647088182411316,
   'lng': -79.35130597436181,
   'labeledLatLngs': [{'label': 'display',
     'lat': 43.647088182411316,
     'lng': -79.35130597436181}],
   'distance': 796,
   'postalCode': 'M5A 1A9',
   'cc': 'CA',
   'city': 'Toronto',
   'state': 'ON',
   'country': 'Canada',
   'formattedAddress': ['33 Villiers St (Villiers St. & Cherry St.)',
    'Toronto ON M5A 1A9',
    'Canada']},
  'categories': [{'id': '4f4528bc4b90abdf24c9de85',
    'name': 'Athletics & Sports',
    'pluralName': 'Athletics & Sports',
    'shortName': 'Athletics & Sports',
    'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/shops/sports_outdoors_',
     'suffix': '.png'},
    'prima

In [72]:
dataframe = json_normalize(items) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories'] + [col for col in dataframe.columns if col.startswith('venue.location.')] + ['venue.id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# filter the category for each row
dataframe_filtered['venue.categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean columns
dataframe_filtered.columns = [col.split('.')[-1] for col in dataframe_filtered.columns]

dataframe_filtered.head(10)

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  """Entry point for launching an IPython kernel.


,name,categories,address,crossStreet,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,neighborhood,id
0,BATLgrounds,Athletics & Sports,33 Villiers St,Villiers St. & Cherry St.,43.647088,-79.351306,"[{'label': 'display', 'lat': 43.64708818241131...",796,M5A 1A9,CA,Toronto,ON,Canada,"[33 Villiers St (Villiers St. & Cherry St.), T...",NaN,52adedf011d24be20f5c4a8f
1,District 28,Coworking Space,28 Logan Ave,NaN,43.655174,-79.340598,"[{'label': 'display', 'lat': 43.65517388160507...",609,NaN,CA,Toronto,ON,Canada,"[28 Logan Ave, Toronto ON, Canada]",Leslieville,5432ce85498e08f48dbefd81
2,Mayfair Racquet and Fitness Clubs,Gym,801 Lake Shore Blvd. E,NaN,43.653707,-79.338742,"[{'label': 'display', 'lat': 43.65370736853231...",513,NaN,CA,Toronto,ON,Canada,"[801 Lake Shore Blvd. E, Toronto ON, Canada]",NaN,5034cd9de4b04e46b78a7038
3,Hearn Generating Station,Government Building,440 Unwin Ave.,Cherry St.,43.646847,-79.335144,"[{'label': 'display', 'lat': 43.64684674617795...",654,M4M 3B9,CA,Toronto,ON,Canada,"[440 Unwin Ave. (Cherry St.), Toronto ON M4M 3...",NaN,4b8cedf5f964a52031e132e3
4,FedEx Ship Centre,Business Service,475 Commissioners St,NaN,43.650679,-79.339094,"[{'label': 'display', 'lat': 43.65067908468494...",265,M4M 1A5,CA,Toronto,ON,Canada,"[475 Commissioners St, Toronto ON M4M 1A5, Can...",NaN,58cd72b502b60e172c6670ac
5,McCleary Park,Baseball Field,Lakeshore Blvd E,at Logan,43.652116,-79.340865,"[{'label': 'display', 'lat': 43.65211581663814...",276,NaN,CA,Toronto,ON,Canada,"[Lakeshore Blvd E (at Logan), Toronto ON, Canada]",NaN,4bd756810b779c74eeee04a0
6,Tim Hortons,Coffee Shop,829 Lakeshore Blvd East,at Carlaw Ave.,43.654367,-79.337359,"[{'label': 'display', 'lat': 43.6543669, 'lng'...",637,M4M 1B2,CA,Toronto,ON,Canada,"[829 Lakeshore Blvd East (at Carlaw Ave.), Tor...",NaN,4c2943d89fb5d13ae7f09a57
7,MountainView Fare trade Coffee,Coffee Shop,26 Logan Ave,Lakeshore Blvd,43.655125,-79.340321,"[{'label': 'display', 'lat': 43.65512520233044...",609,M4M 2M8,CA,Toronto,ON,Canada,"[26 Logan Ave (Lakeshore Blvd), Toronto ON M4M...",NaN,4f317bd7e4b02344c2d2115a
8,Habitat For Humanity ReStore,Department Store,3 Carlaw Avenue,NaN,43.655695,-79.337724,"[{'label': 'display', 'lat': 43.65569451448676...",745,NaN,CA,Toronto,ON,Canada,"[3 Carlaw Avenue, Toronto ON, Canada]",NaN,4dab4e5d0cb6a89c6280f5bd
9,Bayside Rowing Club,Lake,525 Commissioners St.,Carlaw Ave,43.653188,-79.333794,"[{'label': 'display', 'lat': 43.653188, 'lng':...",771,NaN,CA,Toronto,ON,Canada,"[525 Commissioners St. (Carlaw Ave), Toronto O...",NaN,4c9f60ed54c8a1cdd84c964b


In [73]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=15) # generate map centred around Ecco


# add bus stop as a red circle mark
folium.features.CircleMarker(
    [latitude, longitude],
    radius=10,
    popup='Bus stop',
    fill=True,
    color='red',
    fill_color='red',
    fill_opacity=0.6
    ).add_to(venues_map)


# add popular spots to the map as blue circle markers
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=10,
        popup=label,
        fill=True,
        color='blue',
        fill_color='blue',
        fill_opacity=0.6
        ).add_to(venues_map)

# display map
venues_map

In [74]:
dataframe_filtered

,name,categories,address,crossStreet,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,neighborhood,id
0,BATLgrounds,Athletics & Sports,33 Villiers St,Villiers St. & Cherry St.,43.647088,-79.351306,"[{'label': 'display', 'lat': 43.64708818241131...",796,M5A 1A9,CA,Toronto,ON,Canada,"[33 Villiers St (Villiers St. & Cherry St.), T...",NaN,52adedf011d24be20f5c4a8f
1,District 28,Coworking Space,28 Logan Ave,NaN,43.655174,-79.340598,"[{'label': 'display', 'lat': 43.65517388160507...",609,NaN,CA,Toronto,ON,Canada,"[28 Logan Ave, Toronto ON, Canada]",Leslieville,5432ce85498e08f48dbefd81
2,Mayfair Racquet and Fitness Clubs,Gym,801 Lake Shore Blvd. E,NaN,43.653707,-79.338742,"[{'label': 'display', 'lat': 43.65370736853231...",513,NaN,CA,Toronto,ON,Canada,"[801 Lake Shore Blvd. E, Toronto ON, Canada]",NaN,5034cd9de4b04e46b78a7038
3,Hearn Generating Station,Government Building,440 Unwin Ave.,Cherry St.,43.646847,-79.335144,"[{'label': 'display', 'lat': 43.64684674617795...",654,M4M 3B9,CA,Toronto,ON,Canada,"[440 Unwin Ave. (Cherry St.), Toronto ON M4M 3...",NaN,4b8cedf5f964a52031e132e3
4,FedEx Ship Centre,Business Service,475 Commissioners St,NaN,43.650679,-79.339094,"[{'label': 'display', 'lat': 43.65067908468494...",265,M4M 1A5,CA,Toronto,ON,Canada,"[475 Commissioners St, Toronto ON M4M 1A5, Can...",NaN,58cd72b502b60e172c6670ac
5,McCleary Park,Baseball Field,Lakeshore Blvd E,at Logan,43.652116,-79.340865,"[{'label': 'display', 'lat': 43.65211581663814...",276,NaN,CA,Toronto,ON,Canada,"[Lakeshore Blvd E (at Logan), Toronto ON, Canada]",NaN,4bd756810b779c74eeee04a0
6,Tim Hortons,Coffee Shop,829 Lakeshore Blvd East,at Carlaw Ave.,43.654367,-79.337359,"[{'label': 'display', 'lat': 43.6543669, 'lng'...",637,M4M 1B2,CA,Toronto,ON,Canada,"[829 Lakeshore Blvd East (at Carlaw Ave.), Tor...",NaN,4c2943d89fb5d13ae7f09a57
7,MountainView Fare trade Coffee,Coffee Shop,26 Logan Ave,Lakeshore Blvd,43.655125,-79.340321,"[{'label': 'display', 'lat': 43.65512520233044...",609,M4M 2M8,CA,Toronto,ON,Canada,"[26 Logan Ave (Lakeshore Blvd), Toronto ON M4M...",NaN,4f317bd7e4b02344c2d2115a
8,Habitat For Humanity ReStore,Department Store,3 Carlaw Avenue,NaN,43.655695,-79.337724,"[{'label': 'display', 'lat': 43.65569451448676...",745,NaN,CA,Toronto,ON,Canada,"[3 Carlaw Avenue, Toronto ON, Canada]",NaN,4dab4e5d0cb6a89c6280f5bd
9,Bayside Rowing Club,Lake,525 Commissioners St.,Carlaw Ave,43.653188,-79.333794,"[{'label': 'display', 'lat': 43.653188, 'lng':...",771,NaN,CA,Toronto,ON,Canada,"[525 Commissioners St. (Carlaw Ave), Toronto O...",NaN,4c9f60ed54c8a1cdd84c964b
